In [14]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import requests
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

In [15]:
targetSite = 'https://www.starbucks.co.kr:7643/store/getStore.do?r=6AIKXR4SA9'
request = requests.post(targetSite, data={
    'ins_lat': 37.563398,
    'ins_lng': 126.9863309,
    'p_sido_cd': '',
    'p_gugun_cd': '',
    'in_biz_cd': '', 
    'iend': 2000, 
    'set_date': ''
})
starbucks = request.json()
starbucks_df = json_normalize(starbucks, 'list')
starbucks_df.to_csv('./data/starbucks_df.csv', index=False)
starbucks_df

,seq,sido_cd,sido_nm,gugun_cd,gugun_nm,code_order,view_yn,store_num,sido,gugun,...,p01,t05,t30,t36,t27,t29,t43,t48,z9999,p02
0,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
2,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
3,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
4,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1909,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1910,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1911,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1912,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0


In [30]:
starbucks_df_map = starbucks_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'addr', 'doro_address', 'lat', 'lot']]
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)

starbucks_df_map.columns = ['지점이름', '시도코드', '시도이름', '구군코드', '구군이름', '지번주소', '도로명주소', '위도', '경도']

# 지번주소(addr)를 이용해서 '구군', '읍면동' 파생 변수를 추가한다.
starbucks_df_map['구군'] = np.NaN
starbucks_df_map['읍면동'] = np.NaN
addr1 = starbucks_df_map.지번주소.str.split(' ').str[1]
addr2 = starbucks_df_map.지번주소.str.split(' ').str[2]
starbucks_df_map.loc[starbucks_df_map.시도이름 != '세종', '구군'] = addr1
starbucks_df_map.loc[starbucks_df_map.시도이름 != '세종', '읍면동'] = addr2
starbucks_df_map.loc[starbucks_df_map.시도이름 == '세종', '구군'] = ''
starbucks_df_map.loc[starbucks_df_map.시도이름 == '세종', '읍면동'] = addr1

starbucks_df_map.loc[starbucks_df_map.시도이름 == '강원도', '시도이름'] = '강원'

starbucks_df_map.to_csv('./data/starbucks_df_map.csv', index=False)
starbucks_df_map

,지점이름,시도코드,시도이름,구군코드,구군이름,지번주소,도로명주소,위도,경도,구군,읍면동
0,부산P.아울렛,05,부산,0511,기장군,부산광역시 기장군 장안읍 좌천리 497-4,부산광역시 기장군 정관로 1133 (175호),35.325248,129.236612,기장군,장안읍
1,서면NC,05,부산,0504,부산진구,부산광역시 부산진구 전포동 668-1 NC백화점 1층,부산광역시 부산진구 동천로 92 (전포동) NC백화점 1층,35.157036,129.063029,부산진구,전포동
2,BIFF광장,05,부산,0510,중구,부산광역시 중구 남포동3가 1-3,부산광역시 중구 비프광장로 25 (남포동3가),35.098962,129.028053,중구,남포동3가
3,인천공항랜드,07,인천,0712,중구,인천광역시 중구 운서동 2851 인천국제공항 1터미널 지하1층,인천광역시 중구 공항로 271 (운서동),37.448963,126.451472,중구,운서동
4,송파마천사거리DT,01,서울,0114,송파구,서울특별시 송파구 마천동 43-1,서울특별시 송파구 거마로 78 (마천동),37.499829,127.147672,송파구,마천동
...,...,...,...,...,...,...,...,...,...,...,...
1909,호계지식산업센터,08,경기,0819,안양시 동안구,경기도 안양시 동안구 호계동 555-45 호계 데시앙플렉스 데시앙플렉스 118~123호,경기도 안양시 동안구 엘에스로 122 (호계동) 118~123호,37.372351,126.948665,안양시,동안구
1910,안동이마트,11,경북,1123,안동시,경상북도 안동시 옥동 800-3 이마트,경상북도 안동시 옥동1길 2 (옥동),36.558505,128.699308,안동시,옥동
1911,월계역,01,서울,0107,노원구,"서울특별시 노원구 월계동 633-31(104,105,106호)","서울특별시 노원구 초안산로2라길26 월계동(104,105,106호)",37.629500,127.057400,노원구,월계동
1912,광교갤러리아9F,08,경기,0839,수원시 영통구,경기도 수원시 영통구 하동 871-1 9층,경기도 수원시 영통구 광교호수공원로 320 (하동),37.285343,127.057235,수원시,영통구
